## Databases & Python

This is not a tutorial on databases or SQL. If you need that, there are many good tutorials out there on the internet, such as https://www.w3schools.com/sql

This is about how to inter-operate with a database using Python.

Why might you want to do this? Here are a few examples -

- design a web page that accesses the database to display product information
- use a gui to allow users to access the company database on an intranet
- analyse data by retrieving data from a database


#### SQL vs NonSQL databases

Most people think of Relational Databases (RDBMS) when talking of databases. Examples are -
- PostgreSQL
- Oracle
- MS Sql Server
- sqlite3
- MySQL

In recent years an alternative form of storing data has emerged, called NoSQL databases. A few are -
- MongoDB
- CouchDB
- Redis

Here is a brief overview of the differences -

###### RDBMS
- uses a predefined schema
- stores data in tables, rows, and columns
- structure must be set up before it can be used to store data
- difficult to modify schema once it contains any data
- uses a standard query language - SQL
- strict transaction control to ensure data integrity

###### NoSQL
- uses dynamic schema for unstructured data
- data can be stored as documents, key-value pairs, graph structures, and more 
- you can store data without having to define its structure
- each document can have its own unique structure
- each product has its own method for querying the data - no standards
- usually some transaction control, but not as strict

NoSQL databases have become popular, and definitely have their place. However, this workshop will focus on RDBMS databases.

#### DB-API 2.0

Each RDBMS supplier publishes an API allowing it to be accessed programatically. In the early days of Python, different projects were started by different people to write a Python adaptor for each database. They worked, but as they were all different it was difficult to change from one RDBMS supplier to another.

Therefore it was agreed to produce a specification to provide consistency between the various adaptors. The first version, DB-API 1.0, was published in 1996. The second version, DB-API 2.0, was published in 1999. There have been attempts to come up with a revised version since then, but for various reasons they have not been successful. Therefore DB-API 2.0 is the standard on which all current database adaptors are based. The full specification can be read here - https://www.python.org/dev/peps/pep-0249/

Here is a selection of database adaptor modules -

- PostgreSQL - psycopg2
- Sql Server - pyodbc
- sqlite3 - the built-in 'sqlite3' module

The following is not a complete description, but explains the more important aspects of the standard.


#### Connection object

All adaptors support the creation of a Connection object, using a method called connect().

```
import psycopg2
conn = psycopg2.connect(<connection string>)
```

The connection string will vary according the database installation.

Connection objects support the following methods -

- .cursor() returns a Cursor object (see next)

- .commit() commits the current transaction (see Transactions)

- .rollback() rolls back the current transaction (see Transactions)

- .close() closes the connection


#### Cursor object

All adaptors support the creation of a Cursor object.

```
cur = conn.cursor()
```

A cursor object is the means by which we issue SQL statements to the database and get the results.

Cursor objects support the following methods -

- .execute(statement, parameters)
      prepare and execute a SQL statement (query or command)
      parameters can be supplied as a tuple or a dictionary (see Parameters)
- .executemany(statement, sequence of parameters)
      prepare a SQL statement and execute it against each parameter in the sequence
- .fetchone()
      fetch the next row of a result set, or None if no rows available
- .fetchmany(size=cursor.arraysize)
      fetch the next set of rows from the result set, up to a maximum of 'size'
      the set is returned as a list of tuples, or an empty list if no rows available
      size defaults to cursor.arraysize, which can be set as a cursor attribute
- .fetchall()
      fetch all remaining rows from the result set
      the set is returned as a list of tuples, or an empty list if no rows available

Although not specified in DB-API 2.0, most adaptors allow a cursor to return its results in the form of an iterator -

```
cur = conn.cursor()
cur.execute(<statement>)
for row in cur:
    [each row is returned as a tuple]
```


#### Parameters

You may want to execute a query that looks like this -

```
cur.execute(f"SELECT * FROM customers WHERE cust_name = '{customer_name}'")
```

It is <b>strongly</b> recommended that you do not do this. Change the query to this -

```
cur.execute("SELECT * FROM customers WHERE cust_name = ?", (customer_name, ))
```

This is known as a parameterised query. The value for customer_name is replaced by a placeholder, and the value is supplied in the form of a tuple. You can have as many values and placeholders as you like, provided that the tuple contains as many values as there are placeholders. It is also possible to use 'named' parameters and supply the values as a dictionary.

Using parameterised queries is recommended for various reasons -

- if the same query is executed more than once with different customer names, the parameterised version allows the database engine to optimise the query.
- if the value for customer_name comes from an untrusted source (e.g. a web site) the un-parameterised version is vulnerable to SQL injection. The parameterised version is immune to SQL injection.

A complication is that different database adaptors use different placeholders. sqlite3 and pyodbc use '?', psycopg2 uses '%s'. This can make it tricky to write a program that works with different RDBMS's. But see ORMs below.


#### Transactions

All RDBMS's incorporate strict transaction control to ensure data integrity.

If you issue a single SQL command, it will either succeed or fail. But if you issue multiple commands, there is a danger that some will succeed and others will fail. For example. if a bank transfers money from one account to another, and the system fails after the first part of the transfer has executed, the system will be in an inconsistent state.

Transaction control allows you to group multiple commands into a single 'transaction'. The key characteristics are known by the acronym ACID -

- Atomicity - this ensures that either all commands will be successfully executed, as a single unit, or none of them will.
- Consistency - this ensures that a transaction will always bring a database from one valid state to another.
- Isolation - if multiple users try to update the database at the same time, isolation ensures that each transaction is processed independently from any other concurrent transaction
- Durability - this ensures that once a transaction has been committed, it will remain committed even in the case of a system failure.



Standard SQL provides the following commands for transaction control -

- BEGIN or BEGIN TRANSACTION - this tells the database that the following commands must be treated as a unit.
- COMMIT - this tells the database to 'commit' all updates and end the transaction.
- ROLLBACK - this tells the database to 'undo' any updates and return the database to its original state.

When using a Python database adaptor, you should not have to execute any of these commands yourself. (They do allow you to over-ride this if you want to manage transactions yourself, but this is for advanced users only.)

An adaptor is aware of whether a transaction is in progress or not. When you issue any command using cursor.execute(), it will check the state. If a transaction is not in progress, it will silently issue a BEGIN TRANSACTION command. Some adaptors will do this for any command, others will do it for INSERT/UPDATE/DELETE, but not for SELECT. You should not have to worry about this.

It is up to you to decide when to commit or rollback. When you do, it is important that you use the commit() or rollback() methods of the connection object -

```
conn.commit()
or
conn.rollback()
```

It would be a mistake to issue an SQL command like this -

```
cursor.execute('COMMIT')
```

The reason is that the database would correctly commit the transaction, but the adaptor would not be aware of this, so would assume that the transaction was still in progress, and not issue another 'BEGIN' on your next command.


It is common in Python programming to use a context manager when updating a database. Among other benefits, it ensures that a transaction is always committed or rolled back even if an exception occurs. This is an advanced topic that will have to wait for another workshop.


#### ORMs

ORM stands for Object Relational Mapping (as a concept), or Object Relational Mapper (for a particular implementation).

The idea is that, if you are used to dealing with data in an Object Oriented way, switching your brain to dealing with it in a Relational way (rows and columns) can be a challenge. An ORM provides a Mapping between a Relational view and an Object view of the data. This allows a programmer to focus on the data in the way they are most comfortable with, and hide the complexities of having to deal with SQL.

There are a number of ORM's available in the Python world. The most popular are -

- SQLAlchemy
- SQLObject
- Django ORM

A side-benefit (though a very real one) of using an ORM is that you do not have to worry about the differences between the various databases, as this is handled for you under the covers. So you can write your code once and be confident that it will run on any database.


The following examples are taken from the Django ORM documentation.

```
from django.db import models

class Person(models.Model):
    first_name = models.CharField(max_length=30)
    last_name = models.CharField(max_length=30)
```

first_name and last_name are fields of the model. Each field is specified as a class attribute, and each attribute maps to a database column.

The above Person model would create a database table like this:

```
CREATE TABLE myapp_person (
    "id" serial NOT NULL PRIMARY KEY,
    "first_name" varchar(30) NOT NULL,
    "last_name" varchar(30) NOT NULL
);
```


Unfortunately I am not a Django user (or any ORM user in fact) so if you wish to learn more on the subject you will get much better information by reading the documentation (Django's is particularly good) or following one of the many tutorials available on the internet.
